# 크롤링 시작하기

## 함수 시작

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv 
import pandas as pd

In [ ]:
# urldata = pd.read_csv('../data/movie_data/concat.csv', encoding='utf-8')[['영화명','네이버URL']]

In [12]:
urldata = pd.read_csv('../data/movie_data/movie_code_utf8.csv', encoding='utf-8')[['영화명','url_code']]

In [13]:
urldata.head()

,영화명,url_code
0,아바타: 물의 길,74977
1,어벤져스: 에이지 오브 울트론,98438
2,기생충,161967
3,겨울왕국,100931
4,인터스텔라,45290


In [14]:
len(urldata)

280

In [ ]:
# name = []
# code = []

# for i in range(0, len(urldata)):
#     code.append(urldata['네이버URL'][i].split("=")[-1])
#     name.append(urldata['영화명'][i])

In [15]:
m_name = []
m_code = []

for i in range(0, len(urldata)):
    m_code.append(str(urldata['url_code'][i]))
    m_name.append(urldata['영화명'][i])

In [6]:
type(code[0])

str

In [17]:
print(m_name, m_code)

['아바타: 물의 길', '어벤져스: 에이지 오브 울트론', '기생충', '겨울왕국', '인터스텔라', '보헤미안 랩소디', '검사외전', '엑시트', '관상', '아이언맨 3', '설국열차', '캡틴 아메리카: 시빌 워', '수상한 그녀', '해적: 바다로 간 산적', '국가대표', '백두산', '과속스캔들', '탑건: 매버릭', '스파이더맨: 파 프롬 홈', '공조', '트랜스포머 3', '히말라야', '미션임파서블:고스트프로토콜', '스파이더맨: 노 웨이 홈', '트랜스포머: 패자의 역습', '밀정', '최종병기 활', '써니', '한산: 용의 출현', '스파이더맨: 홈 커밍', '1987', '베를린', '마스터', '터널', '어벤져스', '내부자들', '인천상륙작전', '공조2: 인터내셔날', '은밀하게 위대하게', '곡성', '범죄도시', '좋은 놈, 나쁜 놈, 이상한 놈', '늑대소년', '군함도', '미션 임파서블: 폴아웃', '다크 나이트 라이즈', '아저씨', '사도', '전우치', '미션 임파서블: 로그네이션', '킹스맨 : 시크릿 에이전트', '연평해전', '인셉션', '레미제라블', '닥터 스트레인지: 대혼돈의 멀티버스', '캡틴 마블', '쥬라기 월드: 폴른 킹덤', '청년경찰', '숨바꼭질', '덕혜옹주', '더 테러 라이브', '쥬라기 월드', '의형제', '감시자들', '2012', '앤트맨과 와스프', '닥터 스트레인지', '검은 사제들', '안시성', '블랙 팬서', '완득이', '트랜스포머: 사라진 시대', '완벽한 타인', '조커', '월드 워 Z', '타워', '미녀와 야수', '미녀와 야수', '추격자', '독전', '쿵푸팬더 2', '아쿠아맨', '공작', '인사이드 아웃', '킹스맨: 골든 서클', '바람과 함께 사라지다', '마션', '토르: 라그나로크', '어메이징 스파이더맨', '님아, 그 강을 건너지 마오', '조선명탐정 : 각시투구꽃의 비밀', '봉오동 전투', '군도: 민란의 시대', '남산의 부장들',

In [18]:
print(urldata['url_code'][0])

74977


In [19]:
movie_list = m_name
movie_code = m_code

In [21]:
print(movie_code[0])

74977


In [24]:
movie_list = m_name[:10]
movie_code = m_code[:10]

cnt = 10
need_reviews_cnt = cnt
movie_title = []
reviews = []
review_total = []

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for i, code in enumerate(movie_code):
    movie_title.append(movie_list[i])
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    score_total = int(soup.find("div",{"class":"score_total"}).find("em").text.replace(",",""))
    max_range = score_total//10 + 2 if score_total%10 != 0 else score_total//10 + 1
    
    review_data=[]
    for page in range(1,max_range):
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        
        #get : request로 url의  html문서의 내용 요청
        html = requests.get(url)
        #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
        soup = BeautifulSoup(html.content,'html.parser')
        #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
        
        if need_reviews_cnt == 0:
            need_reviews_cnt = cnt
            break

        
        reviews = soup.find_all("div",{"class":"score_reple"}) #리스트로 저장해
        for review in reviews:
            sentence = review.find_all("span")
            
            if sentence[0].text != "관람객":
                review_data.append(sentence[0].text.strip())

            elif sentence[1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                pass
            
            else:
                review_data.append(sentence[1].text.strip())
                # print(sentence[1].text.strip())

            need_reviews_cnt -= 1       # 나중에 빼기
            

        time.sleep(0.5)

    review_total.append(review_data)
    

review_dict = {
    "영화명" : movie_title,
    "리뷰" : review_total
}

review_df = pd.DataFrame(review_dict)
review_df.to_csv('review_utf8.csv', encoding='utf-8')

In [ ]:
movie_list = m_name
movie_code = m_code

cnt = 50
need_reviews_cnt = cnt
movie_title = []
reviews = []
review_total = []

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for i, code in enumerate(movie_code):
    movie_title.append(movie_list[i])
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    score_total = int(soup.find("div",{"class":"score_total"}).find("em").text.replace(",",""))
    max_range = score_total//10 + 2 if score_total%10 != 0 else score_total//10 + 1
    
    review_data=[]
    for page in range(1,max_range):
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        
        #get : request로 url의  html문서의 내용 요청
        html = requests.get(url)
        #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
        soup = BeautifulSoup(html.content,'html.parser')
        #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
        
        if need_reviews_cnt == 0:
            need_reviews_cnt = cnt
            break

        
        reviews = soup.find_all("div",{"class":"score_reple"}) #리스트로 저장해
        for review in reviews:
            sentence = review.find_all("span")
            
            if sentence[0].text != "관람객":
                review_data.append(sentence[0].text.strip())

            elif sentence[1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                # review_data.append(sentence[3].text.strip())
                review_data.append(sentence[3].find("a").get('data-src'))

            else:
                review_data.append(sentence[1].text.strip())
                # print(sentence[1].text.strip())

            need_reviews_cnt -= 1       # 나중에 빼기
            

        time.sleep(0.5)

    review_total.append(review_data)
    

review_dict = {
    "영화명" : movie_title,
    "리뷰" : review_total
}

review_df = pd.DataFrame(review_dict)
review_df.to_csv('review_utf8.csv', encoding='utf-8')

AttributeError: 'NoneType' object has no attribute 'get'

# 리뷰 데이터 전처리

## 라이브러리 설치 및 임포트

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
!pip install kss

In [ ]:
!pip install soynlp

In [1]:
import kss
import re
import pandas as pd
from tqdm import tqdm
from soynlp.normalizer import *
from konlpy.tag import Okt
from pykospacing import Spacing
from kss import split_sentences
from collections import Counter

## 리뷰 데이터 불러오기

In [54]:
# 리뷰 데이터 불러오기
review_data = './review_utf8.csv'
df = pd.read_csv(review_data, encoding='UTF-8', index_col=0)

In [55]:
df.head()

,영화명,리뷰
0,명량,['오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭...
1,극한직업,['일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담없이 진짜 코...
2,신과함께-죄와 벌,"['이정재가 홍보 따라다닌 이유를 알겠음 ㅋㅋㅋ 그리고 관심병사 연기 도랐다', '..."
3,국제시장,['영화내내 가족에게 잘해야겠다 생각이 들었다.우리 아버지는 날위해 모든것을바치신사...
4,어벤져스: 엔드게임,"['마블 3000만큼 사랑합니다', 'Avengers Assemble!', ""I'm..."


In [56]:
df['리뷰'][0]

"['오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네 맛없네 까대며...', '이순신, 그는 신이다.', '군도보다 훨씬 잼있음..극장가서 꼭 보세요', '29일 시사회로 봤어요냉정하게이건 꼭 봐야해..굿', '오늘 아침에 조조로 보고왔습니다. 8월4일에 해군에 입대하는데. 그전에 보기 잘한 영화같습니다. 평소에 영화 많이 좋아하고 자주보는 편인데. 무지 재미있게 봤습니다. 나라 잘지키고 오겠습니다 필승', '정말 재미있네요 잘보고 왔습니다 시사회', '극찬할 수는 없지만 이 정도면 내용, 영상, 연출 전부 꽤나 뛰어난 작품이지 않은가? 평론가들의 평점 6점은 도대체 어떻게 해서 나온 점수인지... 블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이 ...', '국가에 버림을 받아도 의리로 지키려는 충신. 피를 흘리면서 싸우는 군인들의 투지. 두려움을 넘어선 백성들의 의지. 소중한 가족들을 잃은 민족의 한...  슬픈 명작이었습니다....', '보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화ㅠㅠ한 순간도 놓칠 수 없는 해전씬이 특히 압권!!', '작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로 400년전 구국의영웅이 20...']"

In [57]:
# split으로 나누기 
for i in range(len(df)):
    df['리뷰'][i] = df['리뷰'][i].replace('[', '').replace(']', '').split("', '")

In [58]:
df['리뷰'][0]

["'오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네 맛없네 까대며...",
 '이순신, 그는 신이다.',
 '군도보다 훨씬 잼있음..극장가서 꼭 보세요',
 '29일 시사회로 봤어요냉정하게이건 꼭 봐야해..굿',
 '오늘 아침에 조조로 보고왔습니다. 8월4일에 해군에 입대하는데. 그전에 보기 잘한 영화같습니다. 평소에 영화 많이 좋아하고 자주보는 편인데. 무지 재미있게 봤습니다. 나라 잘지키고 오겠습니다 필승',
 '정말 재미있네요 잘보고 왔습니다 시사회',
 '극찬할 수는 없지만 이 정도면 내용, 영상, 연출 전부 꽤나 뛰어난 작품이지 않은가? 평론가들의 평점 6점은 도대체 어떻게 해서 나온 점수인지... 블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이 ...',
 '국가에 버림을 받아도 의리로 지키려는 충신. 피를 흘리면서 싸우는 군인들의 투지. 두려움을 넘어선 백성들의 의지. 소중한 가족들을 잃은 민족의 한...  슬픈 명작이었습니다....',
 '보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화ㅠㅠ한 순간도 놓칠 수 없는 해전씬이 특히 압권!!',
 "작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로 400년전 구국의영웅이 20...'"]

In [59]:
df

,영화명,리뷰
0,명량,['오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭...
1,극한직업,['일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담없이 진짜 코...
2,신과함께-죄와 벌,"['이정재가 홍보 따라다닌 이유를 알겠음 ㅋㅋㅋ 그리고 관심병사 연기 도랐다, 남자..."
3,국제시장,['영화내내 가족에게 잘해야겠다 생각이 들었다.우리 아버지는 날위해 모든것을바치신사...
4,어벤져스: 엔드게임,"['마블 3000만큼 사랑합니다, Avengers Assemble!', ""I'm I..."
5,겨울왕국 2,"['올라프의 1편요약이 기가맥힙니다, 크리스토퍼 뮤비에서 좀 흠칫함, 미래가 보이지..."
6,아바타,"['영화사인 한 획을 그은 영화, 스토리 전개 뻔할지도 몰라요 근데 좋음 스토리 ..."
7,베테랑,"['머야 이거 암살 보다 재미 있네, 황정민, 유해진, 오달수의 연기력도 대단하지만..."
8,괴물,"['뻔한 스토리로 뻔하지 않은 영화를 만들었다, 좀 똑똑한 사람들은 이 영화를 보고..."
9,도둑들,['ㅋㅋㅋ스토리도 탄탄하고 연기자들 연기도 누구 하나 나쁘지 않았는데 왜 저렇게 별...


## 자연어 처리

1. 한 개의 리뷰 안에 여러 개의 문장이 있어 => 문장 토큰화
2. 리뷰에서 영어표현, puncation, 이모티콘은 모두 제거한다. (=한글 표현만 남긴다)
3. 반복되는 문자 정제 (=와하하하하하 같은 것)
4. 띄어쓰기 변환기를 통해 띄어쓰기가 되어있지 않은 문장들을 제대로 변환한다.
5. 형태소 분석기를 통해 각각의 리뷰를 형태소 단위로 분리한다.
6. 한글자로 되어있는 것 제거
7. 빈도수가 많은 순으로 정렬 후 의미없는 단어(불용어) 제거
8. 최소빈도수 이하의 것들은 제거한다.
9. 잘못 분리 된 의미있는 단어들을 찾아서 사용자 단어 사전에 추가 (ex. 디테일 => 디 + 테일로 분리됨)

### 1) 문장 토큰화

In [60]:
# # 문장 토큰화  KSS(Korean Sentence Splitter)

for i in range(len(df)):
    df['리뷰'][i] = kss.split_sentences(df['리뷰'][i])

In [61]:
df.head()

,영화명,리뷰
0,명량,"[['오늘 명량 시사회로 방금 보고 왔음., 단연코 올해의 수작임., 밑에 평론가들..."
1,극한직업,"[['일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요, 오랜만에 부담없이 진짜..."
2,신과함께-죄와 벌,"[['이정재가 홍보 따라다닌 이유를 알겠음 ㅋㅋㅋ, 그리고 관심병사 연기 도랐다],..."
3,국제시장,"[['영화내내 가족에게 잘해야겠다, 생각이 들었다., 우리 아버지는 날위해 모든것을..."
4,어벤져스: 엔드게임,"[['마블 3000만큼 사랑합니다], [Avengers Assemble!', ""I'..."


In [76]:
df['리뷰'][0]

[["'오늘 명량 시사회로 방금 보고 왔음.",
  '단연코 올해의 수작임.',
  '밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네',
  '맛없네',
  '까대며...'],
 ['이순신, 그는 신이다.'],
 ['군도보다 훨씬 잼있음..', '극장가서 꼭 보세요'],
 ['29일 시사회로 봤어요', '냉정하게이건 꼭 봐야해..', '굿'],
 ['오늘 아침에 조조로 보고왔습니다.',
  '8월4일에 해군에 입대하는데.',
  '그전에 보기 잘한 영화같습니다.',
  '평소에 영화 많이 좋아하고 자주보는 편인데.',
  '무지 재미있게 봤습니다.',
  '나라 잘지키고 오겠습니다',
  '필승'],
 ['정말 재미있네요', '잘보고 왔습니다', '시사회'],
 ['극찬할 수는 없지만 이 정도면 내용, 영상, 연출 전부 꽤나 뛰어난 작품이지 않은가?',
  '평론가들의 평점 6점은 도대체 어떻게 해서 나온 점수인지...',
  '블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이 ...'],
 ['국가에 버림을 받아도 의리로 지키려는 충신.',
  '피를 흘리면서 싸우는 군인들의 투지.',
  '두려움을 넘어선 백성들의 의지.',
  '소중한 가족들을 잃은 민족의 한...',
  '슬픈 명작이었습니다....'],
 ['보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화ㅠㅠ한 순간도 놓칠 수 없는 해전씬이 특히 압권!!'],
 ["작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로 400년전 구국의영웅이 20...'"]]

### 2) 한글 표현 남기기

In [87]:
# 정규표현식을 통해 한글 단어만 남기고 모두 제거하기
# def extract_word(re1):
#     hangul = re.compile('[^가-힣]')
#     result = hangul.sub(' ', re1)
#     return result

def extract_word(rev):
    result = []
    for i in range(len(rev)):
        for j in range(len(rev[i])):
            result.append(re.sub('[^가-힣]', " ", rev[i][j]))
    return result

In [82]:
df['리뷰'][0][0]

["'오늘 명량 시사회로 방금 보고 왔음.",
 '단연코 올해의 수작임.',
 '밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네',
 '맛없네',
 '까대며...']

한글 코드 범위   
ㄱ ~ ㅎ: 0x3131 ~ 0x314e   
ㅏ ~ ㅣ: 0x314f ~ 0x3163   
가 ~ 힣: 0xac00 ~ 0xd7a3

In [89]:
# 어떤 식으로 바뀌는 지 확인해보기
print("Before Extraction : ",df['리뷰'][0])
print("After Extraction : ", extract_word(df['리뷰'][0]))

Before Extraction :  [["'오늘 명량 시사회로 방금 보고 왔음.", '단연코 올해의 수작임.', '밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명. 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네', '맛없네', '까대며...'], ['이순신, 그는 신이다.'], ['군도보다 훨씬 잼있음..', '극장가서 꼭 보세요'], ['29일 시사회로 봤어요', '냉정하게이건 꼭 봐야해..', '굿'], ['오늘 아침에 조조로 보고왔습니다.', '8월4일에 해군에 입대하는데.', '그전에 보기 잘한 영화같습니다.', '평소에 영화 많이 좋아하고 자주보는 편인데.', '무지 재미있게 봤습니다.', '나라 잘지키고 오겠습니다', '필승'], ['정말 재미있네요', '잘보고 왔습니다', '시사회'], ['극찬할 수는 없지만 이 정도면 내용, 영상, 연출 전부 꽤나 뛰어난 작품이지 않은가?', '평론가들의 평점 6점은 도대체 어떻게 해서 나온 점수인지...', '블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이 ...'], ['국가에 버림을 받아도 의리로 지키려는 충신.', '피를 흘리면서 싸우는 군인들의 투지.', '두려움을 넘어선 백성들의 의지.', '소중한 가족들을 잃은 민족의 한...', '슬픈 명작이었습니다....'], ['보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화ㅠㅠ한 순간도 놓칠 수 없는 해전씬이 특히 압권!!'], ["작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로 400년전 구국의영웅이 20...'"]]
After Extraction :  [' 오늘 명량 시사회로 방금 보고 왔음 ', '단연코 올해의 수작임 ', '밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명  평론가들 하는 짓이라곤 남이 차린 밥

In [90]:
# 전체 데이터에 대해 위에 함수 실행

df['리뷰'] = df['리뷰'].apply(lambda x:extract_word(x))

In [94]:
df['리뷰'][0]

[' 오늘 명량 시사회로 방금 보고 왔음 ',
 '단연코 올해의 수작임 ',
 '밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명  평론가들 하는 짓이라곤 남이 차린 밥상 맛있네',
 '맛없네',
 '까대며   ',
 '이순신  그는 신이다 ',
 '군도보다 훨씬 잼있음  ',
 '극장가서 꼭 보세요',
 '  일 시사회로 봤어요',
 '냉정하게이건 꼭 봐야해  ',
 '굿',
 '오늘 아침에 조조로 보고왔습니다 ',
 ' 월 일에 해군에 입대하는데 ',
 '그전에 보기 잘한 영화같습니다 ',
 '평소에 영화 많이 좋아하고 자주보는 편인데 ',
 '무지 재미있게 봤습니다 ',
 '나라 잘지키고 오겠습니다',
 '필승',
 '정말 재미있네요',
 '잘보고 왔습니다',
 '시사회',
 '극찬할 수는 없지만 이 정도면 내용  영상  연출 전부 꽤나 뛰어난 작품이지 않은가 ',
 '평론가들의 평점  점은 도대체 어떻게 해서 나온 점수인지   ',
 '블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이    ',
 '국가에 버림을 받아도 의리로 지키려는 충신 ',
 '피를 흘리면서 싸우는 군인들의 투지 ',
 '두려움을 넘어선 백성들의 의지 ',
 '소중한 가족들을 잃은 민족의 한   ',
 '슬픈 명작이었습니다    ',
 '보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화  한 순간도 놓칠 수 없는 해전씬이 특히 압권  ',
 '작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로    년전 구국의영웅이       ']

In [97]:
df['리뷰'][0][0]

' 오늘 명량 시사회로 방금 보고 왔음 '

In [100]:
len(df['리뷰'][0])

31

### 2) 반복된 문자 정제

In [ ]:
# SOYNLP를 이용한 반복되는 문자 정제
for i in range()
for j in range(len(df['리뷰'])):
    review['리뷰'][i] = emoticon_normalize(review['리뷰'][i], num_repeats=4)


TypeError: ord() expected a character, but string of length 21 found

In [96]:
# SOYNLP를 이용한 반복되는 문자 정제
for i in range(len(review\)):
    review['리뷰'][i] = emoticon_normalize(review['리뷰'][i], num_repeats=4)


TypeError: ord() expected a character, but string of length 21 found

In [ ]:
review.head()

### 3) 띄어쓰기 고치기

In [ ]:
# 띄어쓰기 고치기
from pykospacing import Spacing
spacing = Spacing()

for i in range(len(review)):
    review['리뷰'][i] = spacing(review['리뷰'][i]) 

In [ ]:
# 바뀌었는지 전, 후 확인해보기
print("Before Fixing : ",review['리뷰'][0])
print("After Fixing : ", spacing(review['리뷰'][0]))
print("Before Fixing : ",review['리뷰'][1])
print("After Fixing : ", spacing(review['리뷰'][1]))
print("Before Fixing : ",review['리뷰'][2])
print("After Fixing : ", spacing(review['리뷰'][2]))

In [ ]:
review.head()

### 3) 형태소 분석

In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

In [ ]:
# Okt 모델 사용
okt = Okt()
for i in range(0,len(review)):
    review['리뷰'][i] = okt.morphs(review['리뷰'][i],stem=True)
# " ".join(review['리뷰'].tolist())
# words = okt.morphs(words,stem=True)

In [ ]:
review

# 크롤링 코드 연습

In [ ]:
movie_list = ['더 퍼스트 슬램덩크', '스즈메의 문단속']
movie_code = ['223800', '215932']

cnt = 10
need_reviews_cnt = cnt
movie_title = []
reviews = []
review_total = []

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for i, code in enumerate(movie_code):
    movie_title.append(movie_list[i])
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    score_total = int(soup.find("div",{"class":"score_total"}).find("em").text.replace(",",""))
    max_range = score_total//10 + 2 if score_total%10 != 0 else score_total//10 + 1
    
    review_data=[]
    for page in range(1,max_range):
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        #get : request로 url의  html문서의 내용 요청
        html = requests.get(url)
        #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
        soup = BeautifulSoup(html.content,'html.parser')
        #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
        
        if need_reviews_cnt == 0:
            need_reviews_cnt = cnt
            break

        
        reviews = soup.find_all("div",{"class":"score_reple"}) #리스트로 저장해
        for review in reviews:
            sentence = review.find_all("span")
            
            if sentence[0].text != "관람객":
                review_data.append(sentence[0].text.strip())
            elif sentence[1].text == "스포일러가 포함된 감상평입니다. ":
                review_data.append(sentence[2].find('a').get('data-src'))
            else:
                review_data.append(sentence[1].text.strip())
            need_reviews_cnt -= 1       # 나중에 빼기
            print(need_reviews_cnt)
            

        time.sleep(0.5)

    review_total.append(review_data)
    

review_dict = {
    "영화명" : movie_title,
    "리뷰" : review_total
}

review_df = pd.DataFrame(review_dict)
review_df 

#     #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
#     for review in reviews:
#         sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
#         #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
#         if sentence != "":
#             movie = review.find("a",{"class":"movie color_b"}).get_text()
#             score = review.find("em").get_text()
#             review_data.append([movie,sentence,int(score)])
#             need_reviews_cnt-= 1
#     #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지        
#     if need_reviews_cnt < 0:                                         
#         break
#     #다음 페이지를 조회하기 전 0.5초 시간 차를 두기
#     time.sleep(0.5)
     
# columns_name = ["movie","sentence","score"]
# with open ( "samples.csv", "w", newline ="",encoding = 'utf-8' ) as f:
#     write = csv.writer(f)
#     write.writerow(columns_name)
#     write.writerows(review_data)

In [ ]:
# print(movie_title, len(review_data))
# print(review_data[0])
print((review_total[1][-10:]))
print(len(review_total[0]))


# 

# 맨아래